# Sandpit Notebook

For exploring data and testing approaches

## Libraries

In [1]:
import pandas  as pd
import geopandas as gpd
import requests
import os
from datetime import datetime
import numpy as np
from string import digits
import matplotlib.pyplot as plt
import plotly.express as px

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\2031144169.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas  as pd


## Explore ICB Data

In [21]:
FOLDERPATH = "../data/ICB Level CSVs/"
data_icb_all = pd.DataFrame()
for file in os.listdir(FOLDERPATH):
    data = pd.read_csv(FOLDERPATH+file)
    date = file.split(".")[0]
    data["date_str"] = date
    data["date"] = datetime.strptime(date, '%B-%y')
    data_icb_all = pd.concat([data_icb_all, data])

In [22]:
icb_tier3_list = ["NHS Cheshire And Merseyside Integrated Care Board",
                    "NHS Cornwall And The Isles Of Scilly Integrated Care Board",
                    "NHS Devon Integrated Care Board",
                    "NHS Greater Manchester Integrated Care Board",
                    "NHS Kent And Medway Integrated Care Board",
                    "NHS Norfolk And Waveney Integrated Care Board",
                    "NHS North East London Integrated Care Board"]

tier3 =  ["Yes"] * len(icb_tier3_list)

data_icb_all = pd.merge(data_icb_all, pd.Series(tier3, icb_tier3_list, name='tier3'), left_on='System', right_index=True, how='left')
data_icb_all.tier3 = data_icb_all['tier3'].fillna("No")

In [4]:
print(data_icb_all.shape)
print(data_icb_all.columns)
data_icb_all.head(3)

(924, 29)
Index(['Code', 'System', 'Attendances (Type 1)', 'Attendances (Type 2)',
       'Attendances (Type 3)', 'Attendances (Total) ',
       'Attendances in 4 hours or less (Type 1)',
       'Attendances in 4 hours or less (Type 2)',
       'Attendances under 4 hours (Type 3)',
       'Attendances under 4 hours (Total)',
       'Attendances over 4 hours (Type 1)',
       'Attendances over 4 hours (Type 2)',
       'Attendances over 4 hours (Type 3)', 'Attendances over 4 hours (Total)',
       'Percentage in 4 hours or less (Total)',
       'Percentage in 4 hours or less (Type 1)',
       'Percentage in 4 hours or less (Type 2)',
       'Percentage in 4 hours or less (Type 3)',
       'Emergency Admissions via Type 1 A&E',
       'Emergency Admissions via Type 2 A&E',
       'Emergency Admissions via Type 3 and 4 A&E',
       'Total Emergency Admissions via A&E',
       'Other Emergency admissions (i.e not via A&E)',
       'Total Emergency Admissions',
       'Number of patients sp

,Code,System,Attendances (Type 1),Attendances (Type 2),Attendances (Type 3),Attendances (Total),Attendances in 4 hours or less (Type 1),Attendances in 4 hours or less (Type 2),Attendances under 4 hours (Type 3),Attendances under 4 hours (Total),...,Emergency Admissions via Type 2 A&E,Emergency Admissions via Type 3 and 4 A&E,Total Emergency Admissions via A&E,Other Emergency admissions (i.e not via A&E),Total Emergency Admissions,Number of patients spending >4 hours from decision to admit to admission,Number of patients spending >12 hours from decision to admit to admission,date_str,date,tier3
0,QOX,"NHS Bath And North East Somerset, Swindon And ...","16,474",249,"8,983","25,706","9,432",249,"8,855","18,536",...,0,8,"5,318","3,136","8,454","2,070",158,April-22,2022-04-01,False
1,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...","21,440",0,"11,045","32,485","6,097",0,"6,719","12,816",...,0,0,"6,195","2,006","8,201",373,0,April-22,2022-04-01,False
2,QHL,NHS Birmingham And Solihull Integrated Care Board,"36,567",0,"15,427","51,994","21,443",0,"15,427","36,870",...,0,0,"9,699","4,654","14,353","4,983",271,April-22,2022-04-01,False


In [40]:
data_icb_totals = data_icb_all[[
    'Code', 'System', 'Attendances (Total) ',
    'Attendances under 4 hours (Total)', 'Attendances over 4 hours (Total)',
    'Percentage in 4 hours or less (Total)',
    'Total Emergency Admissions via A&E',
    'Other Emergency admissions (i.e not via A&E)',
    'Total Emergency Admissions',
    'Number of patients spending >4 hours from decision to admit to admission',
    'Number of patients spending >12 hours from decision to admit to admission',
    'date_str', 'date', 'tier3'
]]

data_icb_totals.columns = [x.strip() for x in data_icb_totals.columns.tolist()]

data_icb_totals

,Code,System,Attendances (Total),Attendances under 4 hours (Total),Attendances over 4 hours (Total),Percentage in 4 hours or less (Total),Total Emergency Admissions via A&E,Other Emergency admissions (i.e not via A&E),Total Emergency Admissions,Number of patients spending >4 hours from decision to admit to admission,Number of patients spending >12 hours from decision to admit to admission,date_str,date,tier3
0,QOX,"NHS Bath And North East Somerset, Swindon And ...","25,706","18,536","7,170",72.10%,"5,318","3,136","8,454","2,070",158,April-22,2022-04-01,No
1,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...","32,485","12,816","2,099",85.90%,"6,195","2,006","8,201",373,0,April-22,2022-04-01,No
2,QHL,NHS Birmingham And Solihull Integrated Care Board,"51,994","36,870","15,124",70.90%,"9,699","4,654","14,353","4,983",271,April-22,2022-04-01,No
3,QUY,"NHS Bristol, North Somerset And South Gloucest...","27,466","18,061","9,405",65.80%,"6,041","2,509","8,550","2,720","1,169",April-22,2022-04-01,No
4,QU9,"NHS Buckinghamshire, Oxfordshire And Berkshire...","43,735","31,326","12,409",71.60%,"10,163","5,483","15,646","1,804",1,April-22,2022-04-01,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,QJG,NHS Suffolk And North East Essex Integrated Ca...,"33,261","23,784","9,477",71.50%,"6,724","1,401","8,125","2,532","1,180",September-23,2023-09-01,No
38,QXU,NHS Surrey Heartlands Integrated Care Board,"32,981","23,725","9,256",71.90%,"7,784","2,234","10,018","1,959",271,September-23,2023-09-01,No
39,QNX,NHS Sussex Integrated Care Board,"62,845","48,543","14,302",77.20%,"11,483","2,428","13,911","2,536",784,September-23,2023-09-01,No
40,QUA,NHS Black Country Integrated Care Board,"73,634","54,568","19,066",74.10%,"14,895","3,826","18,721","5,185",352,September-23,2023-09-01,No


In [41]:
numeric_cols = [
    'Attendances (Total)',
    'Attendances under 4 hours (Total)', 'Attendances over 4 hours (Total)',
    'Total Emergency Admissions via A&E',
    'Other Emergency admissions (i.e not via A&E)',
    'Total Emergency Admissions',
    'Number of patients spending >4 hours from decision to admit to admission',
    'Number of patients spending >12 hours from decision to admit to admission'
]
percent_col =  'Percentage in 4 hours or less (Total)'
data_icb_totals['Percentage in 4 hours or less (Total)'] = pd.to_numeric(data_icb_totals['Percentage in 4 hours or less (Total)'].apply(lambda x: x[:-1]))


def remove_alphachars(string):
    return ''.join(c for c in str(string) if c in digits)
    
## This breaks the % figure!
for i in numeric_cols:
    data_icb_totals[i] = data_icb_totals[i].apply(remove_alphachars)
    data_icb_totals[i] = pd.to_numeric(data_icb_totals[i])

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\293641350.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\293641350.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\293641350.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [44]:
data_icb_totals.to_csv('../data/cleaned_data.csv', index=False)

In [26]:
tier3_agg = data_icb_totals.groupby(['date', 'tier3'], as_index=False).agg({
    'Attendances (Total)': sum,
    'Attendances under 4 hours (Total)': sum, 
    'Attendances over 4 hours (Total)': sum,
    'System': pd.Series.nunique})

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\3790343292.py:1: FutureWarning:

The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.

C:\Users\spoic\AppData\Local\Temp\ipykernel_10508\3790343292.py:1: FutureWarning:

The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



In [39]:
METRIC = 'Attendances under 4 hours (Total)'
tier3_agg['average'] = tier3_agg[METRIC] / tier3_agg['System']
METRIC2 = 'Attendances under 4 hours (Total)'
tier3_agg['percentage'] = tier3_agg[METRIC2] / tier3_agg['Attendances (Total)'] * 100

fig = px.line(tier3_agg, x="date", y="percentage", color="tier3",
              title=f"% {METRIC2}, Tier 3 vs Rest")
fig.show()

In [37]:
data_icb_totals

,Code,System,Attendances (Total),Attendances under 4 hours (Total),Attendances over 4 hours (Total),Percentage in 4 hours or less (Total),Total Emergency Admissions via A&E,Other Emergency admissions (i.e not via A&E),Total Emergency Admissions,Number of patients spending >4 hours from decision to admit to admission,Number of patients spending >12 hours from decision to admit to admission,date_str,date,tier3,percentage
0,QOX,"NHS Bath And North East Somerset, Swindon And ...",25706,18536,7170,7210,5318,3136,8454,2070,158,April-22,2022-04-01,No,27.892321
1,QHG,"NHS Bedfordshire, Luton And Milton Keynes Inte...",32485,12816,2099,8590,6195,2006,8201,373,0,April-22,2022-04-01,No,6.461444
2,QHL,NHS Birmingham And Solihull Integrated Care Board,51994,36870,15124,7090,9699,4654,14353,4983,271,April-22,2022-04-01,No,29.087972
3,QUY,"NHS Bristol, North Somerset And South Gloucest...",27466,18061,9405,6580,6041,2509,8550,2720,1169,April-22,2022-04-01,No,34.242336
4,QU9,"NHS Buckinghamshire, Oxfordshire And Berkshire...",43735,31326,12409,7160,10163,5483,15646,1804,1,April-22,2022-04-01,No,28.373157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,QJG,NHS Suffolk And North East Essex Integrated Ca...,33261,23784,9477,7150,6724,1401,8125,2532,1180,September-23,2023-09-01,No,28.492829
38,QXU,NHS Surrey Heartlands Integrated Care Board,32981,23725,9256,7190,7784,2234,10018,1959,271,September-23,2023-09-01,No,28.064643
39,QNX,NHS Sussex Integrated Care Board,62845,48543,14302,7720,11483,2428,13911,2536,784,September-23,2023-09-01,No,22.757578
40,QUA,NHS Black Country Integrated Care Board,73634,54568,19066,7410,14895,3826,18721,5185,352,September-23,2023-09-01,No,25.892930


In [ ]:
data_icb_totals['']

## Open Geography Portal API

In [13]:
endpoint_IMD19 = "https://services3.arcgis.com/ivmBBrHfQfDnDf8Q/arcgis/rest/services/Indices_of_Multiple_Deprivation_(IMD)_2019/FeatureServer/0/query"

params = {
    "where": "LADcd = 'E09000022'",
    "outSR": 4326,
    "f": "geoJSON",
    "resultOffset": 0,
    "outFields": "*"
}

_, gdf = request_to_gdf(endpoint_IMD19, params)
gdf.head()

,geometry,FID,lsoa11cd,lsoa11nm,lsoa11nmw,st_areasha,st_lengths,IMD_Rank,IMD_Decile,LSOA01NM,...,OutScore,OutRank,OutDec,TotPop,DepChi,Pop16_59,Pop60_,WorkPop,Shape__Area,Shape__Length
0,"POLYGON ((-0.11479 51.50058, -0.11440 51.49987...",2553,E01003013,Lambeth 036A,Lambeth 036A,343731.285986,4460.477307,21865,7,Lambeth 036A,...,2.183,185,1,1671,125,1395,151,1467.25,885947.224899,7163.535731
1,"POLYGON ((-0.10949 51.49838, -0.11012 51.49716...",2554,E01003014,Lambeth 036B,Lambeth 036B,187536.067480,2836.622325,10007,4,Lambeth 036B,...,2.528,25,1,1656,215,1155,286,1148.00,483318.401237,4554.029028
2,"POLYGON ((-0.11267 51.49550, -0.11242 51.49525...",2555,E01003015,Lambeth 002A,Lambeth 002A,94114.483716,1502.576162,9771,3,Lambeth 002A,...,2.349,68,1,1700,277,1164,259,1169.75,242524.202126,2412.055010
3,"POLYGON ((-0.10890 51.50844, -0.10881 51.50792...",2556,E01003016,Lambeth 036C,Lambeth 036C,161136.163315,2034.085795,20774,7,Lambeth 036C,...,3.310,1,1,2628,127,2336,165,2264.25,415449.422173,3265.328019
4,"POLYGON ((-0.10697 51.50622, -0.10647 51.50506...",2557,E01003017,Lambeth 036D,Lambeth 036D,131261.855190,1684.236054,11736,4,Lambeth 036D,...,2.561,17,1,1928,175,1434,319,1463.00,338398.576820,2704.786761


In [14]:
gdf.explore(column="IMD_Decile", tooltip=['lsoa11nm', 'IMD_Decile', 'IMD_Rank'])